In [11]:
import os
import scipy.io
import numpy as np

# 데이터 폴더 경로 설정
data_folder = '/Users/sihoon/Desktop/Eye_tracking/dataset'

# 빈 리스트를 만들어 데이터와 레이블을 저장합니다.
all_data = []
all_labels = []

# 주어진 폴더 내의 모든 파일을 반복합니다.
for file_name in os.listdir(data_folder):
    # 파일의 확장자가 .mat 인 경우에만 처리합니다.
    if file_name.endswith('.mat'):
        file_path = os.path.join(data_folder, file_name)
        # .mat 파일을 읽어들입니다.
        mat_data = scipy.io.loadmat(file_path)

        # 데이터와 레이블을 추출하여 저장합니다.
        # 데이터 추출: 'ETdata' 내의 'pos'에서 3번째와 4번째 열을 가져옵니다.
        data = mat_data['ETdata']['pos'][0, 0][:, [3, 4]]
        # 레이블 추출: 'ETdata' 내의 'pos'에서 5번째 열을 가져온 후 반올림하여 정수형으로 변환합니다.
        labels = mat_data['ETdata']['pos'][0, 0][:, 5].round().astype(int)

        # 데이터와 레이블을 리스트에 추가합니다.
        all_data.append(data)
        all_labels.append(labels)

# 데이터와 레이블을 수직으로 병합합니다.
merged_data = np.vstack(all_data)
merged_labels = np.hstack(all_labels)

# 데이터와 레이블을 병합한 후에 0이 포함된 행을 제거하고 인덱스를 초기화합니다.
zero_indices = np.where(np.any(merged_data == 0, axis=1))[0]
merged_data = np.delete(merged_data, zero_indices, axis=0)
merged_labels = np.delete(merged_labels, zero_indices, axis=0)

data = merged_data
label = merged_labels - 1

# 유일한 레이블 값 출력
unique_labels = np.unique(label)
print(unique_labels)

print("불러온 데이터 배열 형태:", data.shape)
print("데이터의 차원 수:", data.ndim)

[0 1 2 3 4 5]
불러온 데이터 배열 형태: (38464, 2)
데이터의 차원 수: 2


In [5]:
# 데이터를 현재값 - 이전값으로 변환
data_diff = np.diff(data, axis=0)

label_cut = label[1:]

# 데이터와 레이블 확인
print("변환된 데이터 배열 형태:", data_diff.shape)
print("변환된 데이터 예시:")
print(data_diff[:5])
print("생성된 레이블 배열 형태:", label_cut.shape)
print("생성된 레이블 예시:")
print(label_cut[:5])

변환된 데이터 배열 형태: (38463, 2)
변환된 데이터 예시:
[[ 0.5864 -0.458 ]
 [ 0.4093  0.5125]
 [ 0.3036 -0.7235]
 [ 0.3631  0.1704]
 [ 0.8035 -0.0751]]
생성된 레이블 배열 형태: (38463,)
생성된 레이블 예시:
[0 0 0 0 0]


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 준비
x = data_diff
y = label_cut

# 데이터 분할 (트레이닝 데이터셋과 테스트 데이터셋)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# LSTM 입력 데이터의 형태를 맞추기 위해 reshape
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

# 클래스 수 설정
num_classes = 6

# 모델 정의
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.1))
model.add(LSTM(32))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 조기 종료 콜백 정의
# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

# 모델 훈련
history = model.fit(x_train, y_train, epochs=500, batch_size=32, validation_split=0.2, verbose=2)

# 모델 평가
accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print("Accuracy:", accuracy)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 1, 64)             17152     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 1, 64)             0         
                                                                 
 dropout_2 (Dropout)         (None, 1, 64)             0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 32)                0         
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 32)               

In [7]:
# 테스트 데이터에 대한 예측
predicted_probabilities = model.predict(x_test)
predicted_classes = np.argmax(predicted_probabilities, axis=1)

# 예측된 클래스와 실제 레이블을 순서쌍으로 출력
print("Predicted | Actual")
print("-------------------")
for pred, actual in zip(predicted_classes, y_test):
    print(f"   {pred}      |    {actual}")


241/241 [==============================] - 0s 427us/step
Predicted | Actual
-------------------
   0      |    0
   0      |    0
   1      |    1
   0      |    0
   0      |    0
   1      |    1
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   2      |    3
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   1      |    1
   1      |    2
   2      |    1
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   4      |    4
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   2      |    1
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   1      |    1
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   0      |    0
   

In [8]:
# 일치하지 않는 예측 인덱스 찾기
incorrect_indices = np.where(predicted_classes != y_test)[0]

# 일치하지 않는 예측 출력
print("Incorrect Predictions:")
print("Predicted | Actual")
print("-------------------")
for index in incorrect_indices:
    pred = predicted_classes[index]
    actual = y_test[index]
    print(f"   {pred}      |    {actual}")


Incorrect Predictions:
Predicted | Actual
-------------------
   2      |    3
   1      |    2
   2      |    1
   2      |    1
   0      |    2
   1      |    4
   0      |    2
   1      |    4
   2      |    1
   1      |    2
   0      |    2
   0      |    3
   1      |    2
   2      |    1
   0      |    2
   1      |    2
   1      |    2
   2      |    1
   1      |    2
   1      |    0
   2      |    1
   0      |    2
   1      |    0
   0      |    3
   0      |    1
   0      |    3
   2      |    1
   1      |    2
   2      |    1
   0      |    1
   0      |    1
   0      |    1
   0      |    2
   1      |    2
   0      |    1
   2      |    0
   1      |    2
   0      |    2
   1      |    2
   0      |    1
   0      |    3
   2      |    1
   0      |    3
   2      |    1
   0      |    1
   0      |    1
   0      |    2
   2      |    0
   0      |    2
   2      |    1
   2      |    0
   2      |    1
   0      |    2
   2      |    0
   0      |    1
   

In [9]:
# # 모델 저장
# model.save('//Users/sihoon/Desktop/Eye_tracking/eye_model.h5')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
